In [ ]:
from transformers import BartForConditionalGeneration, BartTokenizer

model = BartForConditionalGeneration.from_pretrained('/content/drive/MyDrive/bart-asset-lora')
tokenizer = BartTokenizer.from_pretrained('/content/drive/MyDrive/bart-asset-lora')

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/558M [00:00<?, ?B/s]

In [ ]:
import json

with open("asset_test.json") as f:  # or your test file
    test_data = json.load(f)

In [ ]:
inputs = tokenizer(
    test_data[0]["original"],  # single example
    return_tensors="pt",       # PyTorch tensors
    truncation=True,
    max_length=64
)

In [ ]:
outputs = model.generate(
    **inputs,
    max_length=64,
    num_beams=4,        # beam search for better quality
    early_stopping=True
)

simplified_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
print("Original:", test_data[0]["original"])
print("Simplified:", simplified_text)

Original: One side of the armed conflicts is composed mainly of the Sudanese military and the Janjaweed, a Sudanese militia group recruited mostly from the Afro-Arab Abbala tribes of the northern Rizeigat region in Sudan.
Simplified: One side of the armed conflicts is composed of the Sudanese military and Janjaweed, a Sudanese militia group recruited mostly from the Afro-Arab Abbala tribes of the northern Rizeigat region in Sudan.


In [ ]:
for item in test_data[:5]:  # first 5 examples
    inputs = tokenizer(item["original"], return_tensors="pt", truncation=True, max_length=64)
    outputs = model.generate(**inputs, max_length=64, num_beams=4, early_stopping=True)
    simplified = tokenizer.decode(outputs[0], skip_special_tokens=True)
    print("Original:", item["original"])
    print("Simplified:", simplified)
    print("-" * 80)

Original: One side of the armed conflicts is composed mainly of the Sudanese military and the Janjaweed, a Sudanese militia group recruited mostly from the Afro-Arab Abbala tribes of the northern Rizeigat region in Sudan.
Simplified: One side of the armed conflicts is composed of the Sudanese military and Janjaweed, a Sudanese militia group recruited mostly from the Afro-Arab Abbala tribes of the northern Rizeigat region in Sudan.
--------------------------------------------------------------------------------
Original: Jeddah is the principal gateway to Mecca, Islam's holiest city, which able-bodied Muslims are required to visit at least once in their lifetime.
Simplified: Jeddah is the main gateway to Mecca, Islam's holiest city. Muslims are required to visit at least once in their lifetime.
--------------------------------------------------------------------------------
Original: The Great Dark Spot is thought to represent a hole in the methane cloud deck of Neptune.
Simplified: The

In [1]:
import json
import re
import random
import numpy as np
import torch
from tqdm import tqdm

from transformers import (
    BartForConditionalGeneration,
    BartTokenizer,
    GenerationConfig
)

import nltk
from nltk.corpus import cmudict
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction

nltk.download("cmudict")

[nltk_data] Downloading package cmudict to /root/nltk_data...
[nltk_data]   Unzipping corpora/cmudict.zip.


True

In [5]:
MODEL_DIR = "/content/drive/MyDrive/bart-asset-lora"
TEST_JSON = "asset_test.json"

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

tokenizer = BartTokenizer.from_pretrained(MODEL_DIR)
model = BartForConditionalGeneration.from_pretrained(MODEL_DIR).to(device)
model.eval()

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/558M [00:00<?, ?B/s]

BartForConditionalGeneration(
  (model): BartModel(
    (shared): BartScaledWordEmbedding(50265, 768, padding_idx=1)
    (encoder): BartEncoder(
      (embed_tokens): BartScaledWordEmbedding(50265, 768, padding_idx=1)
      (embed_positions): BartLearnedPositionalEmbedding(1026, 768)
      (layers): ModuleList(
        (0-5): 6 x BartEncoderLayer(
          (self_attn): BartAttention(
            (k_proj): lora.Linear(
              (base_layer): Linear(in_features=768, out_features=768, bias=True)
              (lora_dropout): ModuleDict(
                (default): Dropout(p=0.1, inplace=False)
              )
              (lora_A): ModuleDict(
                (default): Linear(in_features=768, out_features=16, bias=False)
              )
              (lora_B): ModuleDict(
                (default): Linear(in_features=16, out_features=768, bias=False)
              )
              (lora_embedding_A): ParameterDict()
              (lora_embedding_B): ParameterDict()
              (lo

In [23]:
# ============================================================
# 1. Load model and tokenizer
# ============================================================
MODEL_DIR = '/content/drive/MyDrive/bart-asset-lora'

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

tokenizer = BartTokenizer.from_pretrained(MODEL_DIR)
model = BartForConditionalGeneration.from_pretrained(MODEL_DIR)
model = model.to(device)
model.eval()

gen_config = GenerationConfig.from_model_config(model.config)
gen_config.use_cache = False
gen_config.num_beams = 4
gen_config.max_new_tokens = 64
gen_config.early_stopping = True
gen_config.forced_bos_token_id = 0
gen_config.no_repeat_ngram_size = 3
gen_config.length_penalty = 1.1

# ============================================================
# 2. Load test dataset
# ============================================================
TEST_JSON = "/content/synthetic_test.json"  # Replace with your file path

with open(TEST_JSON, "r") as f:
    eval_data = json.load(f)

random.seed(1)
eval_data = random.sample(eval_data, int(0.1 * len(eval_data)))

print("Evaluation samples:", len(eval_data))


# ============================================================
# 3. Evaluation metric functions
# ============================================================

# ---------- Syllable counting ----------
d = cmudict.dict()

def count_syllables(word):
    word = word.lower()
    if word in d:
        return max([len([y for y in x if y[-1].isdigit()]) for x in d[word]])
    else:
        # Fallback
        count = 0
        vowels = 'aeiouy'
        prev_vowel = False
        for c in word:
            is_vowel = c in vowels
            if is_vowel and not prev_vowel:
                count += 1
            prev_vowel = is_vowel
        if word.endswith('e'):
            count -= 1
        return max(count, 1)

# ---------- Flesch-Kincaid ----------
def flesch_kincaid_grade(text):
    sentences = re.split(r'[.!?]+', text)
    sentences = [s.strip() for s in sentences if s.strip()]
    words = re.findall(r'\b\w+\b', text.lower())
    if not sentences or not words:
        return 0.0
    total_syllables = sum(count_syllables(w) for w in words)
    grade = 0.39 * (len(words)/len(sentences)) + 11.8*(total_syllables/len(words)) - 15.59
    return round(grade, 2)

# ---------- BLEU ----------
def bleu_score(reference, candidate):
    ref_tokens = re.findall(r'\b\w+\b', reference.lower())
    cand_tokens = re.findall(r'\b\w+\b', candidate.lower())
    smoothing = SmoothingFunction().method1
    score = sentence_bleu([ref_tokens], cand_tokens, smoothing_function=smoothing)
    return round(score, 4)

# ---------- SARI approximation ----------
def sari_score(source, reference, candidate):
    source_tokens = set(re.findall(r'\b\w+\b', source.lower()))
    ref_tokens = set(re.findall(r'\b\w+\b', reference.lower()))
    cand_tokens = set(re.findall(r'\b\w+\b', candidate.lower()))

    # Added
    added = cand_tokens - source_tokens
    add_prec = len(added & ref_tokens) / len(added) if added else 0.0

    # Kept
    kept = source_tokens & cand_tokens
    ref_kept = source_tokens & ref_tokens
    keep_prec = len(kept & ref_kept) / len(kept) if kept else 0
    keep_rec = len(kept & ref_kept) / len(ref_kept) if ref_kept else 0
    keep_f1 = 2*keep_prec*keep_rec/(keep_prec+keep_rec) if (keep_prec+keep_rec)>0 else 0

    # Deleted
    deleted = source_tokens - cand_tokens
    ref_deleted = source_tokens - ref_tokens
    del_prec = len(deleted & ref_deleted)/len(deleted) if deleted else 0.0

    sari = (add_prec + keep_f1 + del_prec)/3*100
    return round(sari, 2)

# ---------- Compression ----------
def compression_ratio(original, simplified):
    char_ratio = len(simplified)/len(original) if len(original)>0 else 0
    orig_words = len(re.findall(r'\b\w+\b', original))
    simp_words = len(re.findall(r'\b\w+\b', simplified))
    word_ratio = simp_words / orig_words if orig_words>0 else 0
    return {
        'char_ratio': round(char_ratio, 4),
        'word_ratio': round(word_ratio, 4),
        'char_reduction_pct': round((1-char_ratio)*100, 2),
        'word_reduction_pct': round((1-word_ratio)*100, 2)
    }

# ---------- Sentence length ----------
def average_sentence_length(text):
    sentences = [s.strip() for s in re.split(r'[.!?]+', text) if s.strip()]
    if not sentences:
        return {'avg_sentence_length':0.0, 'total_sentences':0}
    total_words = sum(len(re.findall(r'\b\w+\b', s)) for s in sentences)
    avg_len = total_words/len(sentences)
    return {'avg_sentence_length': round(avg_len,2), 'total_sentences':len(sentences)}

# ============================================================
# 4. Generate predictions
# ============================================================
def generate_predictions(data, model, tokenizer, device, max_source_len=64):
    preds = []
    for item in tqdm(data, desc="Generating predictions"):
        src = item["original"].strip()
        enc = tokenizer(src, return_tensors="pt", truncation=True, max_length=max_source_len)
        enc = {k:v.to(device) for k,v in enc.items()}
        with torch.no_grad():
            out_ids = model.generate(**enc, generation_config=gen_config)
        pred = tokenizer.decode(out_ids[0], skip_special_tokens=True)
        preds.append(pred)
    return preds

# ============================================================
# 5. Calculate metrics
# ============================================================
def calculate_dataset_metrics(data, preds):
    fk_orig, fk_pred = [], []
    bleu_scores, sari_scores = [], []
    comp_word, comp_char = [], []
    asl_orig, asl_pred = [], []

    for item, pred in tqdm(zip(data, preds), total=len(preds), desc="Scoring metrics"):
        original = item["original"]
        if "simplifications" in item and item["simplifications"]:
            references = item["simplifications"]
        else:
            references = [item.get("simplified","")]

        # FK
        fk_orig.append(flesch_kincaid_grade(original))
        fk_pred.append(flesch_kincaid_grade(pred))

        # BLEU
        bleu_per_refs = [bleu_score(ref, pred) for ref in references if ref.strip()]
        bleu_scores.append(max(bleu_per_refs) if bleu_per_refs else 0.0)

        # SARI
        sari_per_refs = [sari_score(original, ref, pred) for ref in references if ref.strip()]
        sari_scores.append(np.mean(sari_per_refs) if sari_per_refs else 0.0)

        # Compression
        comp = compression_ratio(original, pred)
        comp_word.append(comp['word_ratio'])
        comp_char.append(comp['char_ratio'])

        # Sentence length
        asl_orig.append(average_sentence_length(original)['avg_sentence_length'])
        asl_pred.append(average_sentence_length(pred)['avg_sentence_length'])

    results = {
        'dataset_size': len(preds),

        'flesch_kincaid': {
            'original': {'mean': round(float(np.mean(fk_orig)),2),
                         'std': round(float(np.std(fk_orig)),2),
                         'median': round(float(np.median(fk_orig)),2)},
            'pred': {'mean': round(float(np.mean(fk_pred)),2),
                     'std': round(float(np.std(fk_pred)),2),
                     'median': round(float(np.median(fk_pred)),2)},
            'improvement': round(float(np.mean(fk_orig)-np.mean(fk_pred)),2)
        },

        'bleu_max_over_refs': {
            'mean': round(float(np.mean(bleu_scores)),4),
            'std': round(float(np.std(bleu_scores)),4),
            'median': round(float(np.median(bleu_scores)),4)
        },

        'sari': {
            'mean': round(float(np.mean(sari_scores)),2),
            'std': round(float(np.std(sari_scores)),2),
            'median': round(float(np.median(sari_scores)),2)
        },

        'compression': {
            'word_ratio': {
                'mean': round(float(np.mean(comp_word)),4),
                'std': round(float(np.std(comp_word)),4),
                'median': round(float(np.median(comp_word)),4)
            },
            'char_ratio': {
                'mean': round(float(np.mean(comp_char)),4),
                'std': round(float(np.std(comp_char)),4),
                'median': round(float(np.median(comp_char)),4)
            },
            'avg_word_reduction_pct': round((1-float(np.mean(comp_word)))*100,2)
        },

        'avg_sentence_length': {
            'original': {'mean': round(float(np.mean(asl_orig)),2),
                         'std': round(float(np.std(asl_orig)),2),
                         'median': round(float(np.median(asl_orig)),2)},
            'pred': {'mean': round(float(np.mean(asl_pred)),2),
                     'std': round(float(np.std(asl_pred)),2),
                     'median': round(float(np.median(asl_pred)),2)},
            'reduction': round(float(np.mean(asl_orig)-np.mean(asl_pred)),2)
        }
    }

    return results

# ============================================================
# 6. Run evaluation
# ============================================================
preds = generate_predictions(eval_data, model, tokenizer, device)
results = calculate_dataset_metrics(eval_data, preds)

OUT_PATH = "/content/drive/MyDrive/simplification_metrics.json"
with open(OUT_PATH, "w") as f:
    json.dump(results, f, indent=2)

print("Saved results to:", OUT_PATH)
results

Evaluation samples: 447


Scoring metrics: 100%|██████████| 447/447 [00:00<00:00, 2466.18it/s]


Saved results to: /content/drive/MyDrive/simplification_metrics.json


{'dataset_size': 447,
 'flesch_kincaid': {'original': {'mean': 21.89, 'std': 2.69, 'median': 21.8},
  'pred': {'mean': 12.13, 'std': 2.86, 'median': 11.75},
  'improvement': 9.76},
 'bleu_max_over_refs': {'mean': 0.3595, 'std': 0.2306, 'median': 0.3475},
 'sari': {'mean': 75.77, 'std': 12.97, 'median': 76.67},
 'compression': {'word_ratio': {'mean': 0.588,
   'std': 0.1187,
   'median': 0.5714},
  'char_ratio': {'mean': 0.5209, 'std': 0.1033, 'median': 0.5143},
  'avg_word_reduction_pct': 41.2},
 'avg_sentence_length': {'original': {'mean': 31.22,
   'std': 5.17,
   'median': 31.0},
  'pred': {'mean': 18.21, 'std': 4.25, 'median': 18.0},
  'reduction': 13.01}}